# YOLO : Training and Testingary imports


#### General Steps to Follow

1. Importing Packages.
2. Training Object Detection Model with YOLO.
3. Testing.

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

In [3]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator
import matplotlib.pyplot as plt
from PIL import Image

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

##  2) Training Object Detection Model with YOLO

In the training phase, I began by obtaining data from Roboflow, distributed across separate files. To streamline the process, I consolidated these files and created a YAML configuration file. This YAML file encapsulates crucial details for training, such as specifying the number of classes—one in this case—and their corresponding names (Helmets). Leveraging YOLO (You Only Look Once), a powerful object detection framework, I initiated the training process which incorporates essential parameters such as the configuration file (data.yaml), training epochs, batch size, image size, device specification, and optimization settings. The process involves iterating through 200 epochs with a batch size of 2, optimizing the model to detect objects within the specified classes. The YAML file serves as a key reference, ensuring the model comprehensively recognizes helmets during training, thereby enhancing its object detection capabilities.

In [2]:
!yolo task=detect mode=train model=yolov8n.pt data="data.yaml" epochs=50 batch=16 imgsz=320 device=cpu workers=2 optimizer=Adam pretrained=true val=true plots=true save=True show=true optimize=true lr0=0.001 lrf=0.01 fliplr=0.0 patience=0 amp=False

^C


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

##  3) Testing

Subsequent to the training phase, I rigorously assessed the model's performance through comprehensive testing. Utilizing a diverse set of evaluation criteria, I subjected the trained YOLO model to videos and images to ensure its robustness and accuracy. The model's effectiveness was evaluated against two distinct scenarios, as exemplified by the following codes

In [4]:
model = YOLO('best.pt')

In [6]:
image_path = "test.jpg"
image = cv2.imread(f"{image_path}")

# Get the predictions from the YOLOv8 model
results = model.predict(image, show_conf=True)


0: 640x384 2 glovess, 1 Helmets, 1 Vests, 339.0ms
Speed: 18.0ms preprocess, 339.0ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 384)


In [8]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'gloves', 1: 'Googles', 2: 'Helmets', 3: 'Vests', 4: '1'}
 orig_img: array([[[170, 166, 165],
         [170, 166, 165],
         [171, 167, 166],
         ...,
         [213, 211, 211],
         [219, 214, 215],
         [214, 209, 210]],
 
        [[170, 166, 165],
         [170, 166, 165],
         [171, 167, 166],
         ...,
         [212, 210, 210],
         [219, 214, 215],
         [214, 209, 210]],
 
        [[170, 166, 165],
         [170, 166, 165],
         [171, 167, 166],
         ...,
         [212, 210, 210],
         [218, 213, 214],
         [213, 208, 209]],
 
        ...,
 
        [[197, 201, 202],
         [197, 201, 202],
         [197, 200, 204],
         ...,
         [179, 184, 187],
         [179, 184, 187],
         [178, 183, 186]],
 
        [[196, 200, 201],
         [194, 198, 199],
         [194, 197, 2

In [11]:
for r in results:
    print(r.boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([3., 0., 2., 0.])
conf: tensor([0.9738, 0.9493, 0.9489, 0.9391])
data: tensor([[2.0540e+02, 5.2754e+02, 6.1580e+02, 9.8800e+02, 9.7382e-01, 3.0000e+00],
        [5.8556e+02, 8.9475e+02, 6.5353e+02, 9.9552e+02, 9.4933e-01, 0.0000e+00],
        [3.0594e+02, 3.3837e+02, 4.5176e+02, 4.4035e+02, 9.4890e-01, 2.0000e+00],
        [1.8379e+02, 8.8623e+02, 2.4005e+02, 1.0116e+03, 9.3909e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (1280, 720)
shape: torch.Size([4, 6])
xywh: tensor([[410.6036, 757.7675, 410.4026, 460.4646],
        [619.5430, 945.1392,  67.9731, 100.7693],
        [378.8503, 389.3613, 145.8144, 101.9756],
        [211.9239, 948.9354,  56.2610, 125.4042]])
xywhn: tensor([[0.5703, 0.5920, 0.5700, 0.3597],
        [0.8605, 0.7384, 0.0944, 0.0787],
        [0.5262, 0.3042, 0.2025, 0.0797],
        [0.2943, 0.7414, 0.0781, 0.0980]])
xyxy: tensor([[ 205.4023,  527.5352,  615.8049,  987.9998],
        [ 58

In [20]:
df = results[0].boxes.xyxy.numpy()

In [21]:
len(df)

4

In [19]:
results[0].boxes.cls.numpy()

array([          3,           0,           2,           0], dtype=float32)

In [ ]:


# Code for testing on images
model = YOLO('best.pt')
image_path = "test.jpg"
image = cv2.imread(f"{image_path}")

# Get the predictions from the YOLOv8 model
results = model.predict(image, show_conf=True)

# Display image before predictions
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')

# Apply bounding boxes and display image after predictions
for r in results:
    annotator = Annotator(image)
    boxes = r.boxes
    for box in boxes:
        b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
        c = box.cls
        x = box
        annotator.box_label(b, model.names[int(c)])

image_with_boxes = annotator.result()

cv2.imshow('res', cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
cv2.waitKey(0)


0: 640x384 2 glovess, 1 Helmets, 1 Vests, 207.0ms
Speed: 7.0ms preprocess, 207.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)


-1

In [ ]:
# Code for testing on videos
model = YOLO('best.pt')
cap = cv2.VideoCapture("1.mp4")
cap.set(3, 640)
cap.set(4, 480)

while cap.isOpened():
    _, frame = cap.read()
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = model.predict(img)

    for r in results:
        
        annotator = Annotator(frame)
        
        boxes = r.boxes
        for box in boxes:
            if(box.conf > 0.6):
                b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
                c = box.cls
                annotator.box_label(b, model.names[int(c)])
          
    frame = annotator.result()  
    cv2.imshow('YOLO V8 Detection', frame)     
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 93.2ms
Speed: 58.6ms preprocess, 93.2ms inference, 239.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 8.0ms
Speed: 1.0ms preprocess, 8.0ms in

In [22]:
import numpy as np

# Create a NumPy array
arr = np.array([1, 0, 2, 0, 0, 3, 4, 0])

# Get the count of zeros in the array
count_zeros = np.count_nonzero(arr == 0)

print(f"The number of zeros in the array is: {count_zeros}")

The number of zeros in the array is: 4


NameError: name 'frame' is not defined

In [65]:
image_path = "test.jpg"
image = cv2.imread(f"{image_path}")

In [66]:
# loading the trained custom model
model = YOLO('best.pt')
model.conf = 0.5  # NMS confidence threshold

model2 = YOLO('yolov8n.pt')
model2.conf = 0.5  # NMS confidence threshold
model2.classes = [0]  # person detection

In [67]:
results = model.predict(image, show_conf=True)

bbx = results[0].boxes.xyxy.numpy()
cls1 = results[0].boxes.cls.numpy()

results2 = model2.predict(image, show_conf=True)

bbx2 = results2[0].boxes.xyxy.numpy()
cls2 = results2[0].boxes.cls.numpy()
num_persons = np.count_nonzero(cls2 == 0.0)    
num_gloves = np.count_nonzero(cls1 == 0.0)   
num_googles = np.count_nonzero(cls1 == 1.0)
num_helmets = np.count_nonzero(cls1 == 2.0)
num_vests = np.count_nonzero(cls1 == 3.0)


0: 640x384 2 glovess, 1 Helmets, 1 Vests, 385.0ms
Speed: 10.0ms preprocess, 385.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 214.0ms
Speed: 6.0ms preprocess, 214.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)


In [68]:
num_persons, num_gloves, num_googles, num_helmets, num_vests

(1, 2, 0, 1, 1)

In [69]:
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent / 100)
    height = int(frame.shape[0] * percent / 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

In [70]:
# Apply bounding boxes and display image after predictions
for r in results:
    annotator = Annotator(image)
    boxes = r.boxes
    for box in boxes:
        b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
        c = box.cls
        x = box
        annotator.box_label(b, model.names[int(c)])

image_with_boxes = annotator.result()



In [71]:
# Apply bounding boxes and display image after predictions
for r in results2:
    annotator = Annotator(image_with_boxes)
    boxes = r.boxes
    for box in boxes:
        b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
        c = box.cls
        x = box
        annotator.box_label(b, model2.names[int(c)])

image_with_boxes = annotator.result()



In [72]:
image_with_boxes = rescale_frame(image_with_boxes, 60)
cv2.imshow('res', image_with_boxes)
cv2.waitKey(0)

93